In [ ]:
%pip install pandas numpy python-calamine openpyxl -U --quiet

In [ ]:
import pandas as pd
import numpy as np
import re

load = "KnownSites_manualedit.xlsx"
sink = "KnownSites_normalized.csv"

In [ ]:
df = pd.read_excel(load, engine='calamine')
unnamed = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed)
df.head()

In [ ]:
def normalize(string: str):
    if re.match(r"\d+\.\d+\s*[Ne]*,\s*\d+\.\d+\s*[Ee]*", string.strip()):
        temp = string.upper().replace("N", "").replace("E", "").split(",")
        return tuple(map(float, temp))
    
    elif re.match(r"""\d+[°]\s*\d*[']*\s*\d*[.]*\d*["']*\s*[Nn]\s*[,]{1}\s*\d+[°]\s*\d*[']*\s*\d*["']*[.]*\d*\s*[Ee]*""", string.strip()):
        temp = [re.split('[°\'"]', i) for i in string.split(",")]
        
        return tuple(map(convert, temp))
    
    else:
        return (0, 0)
    
def convert(values: list):
    values = [i.strip() for i in values if i.strip() != ""]
    total = 0
    for i, value in enumerate(values):
        if value.lower() in ["w", "s"]:
            total *= -1
            break
        elif value.lower() in ["e", "n"]:
            break
        if i == 0:
            total += float(value.strip())
        elif i == 1:
            total += float(value)/60
        elif i == 2:
            total += float(value)/3600
    
    return total
        

test = df.copy()
normalized = test['COORDINATES'].apply(normalize)
test.loc[:,'coord_ns'], test.loc[:,'coord_ew'] = list(zip(*normalized))[0], list(zip(*normalized))[1]
test

In [ ]:
fail = test[(test['coord_ns'] == 0) & (test['coord_ew'] == 0)]
correct = test[(test['coord_ns'] != 0) & (test['coord_ew'] != 0)]

fail

In [ ]:
correct.describe()

In [ ]:
correct.to_excel("KnownSites_normalized.xlsx", index=False)